In [2]:
# @title Mount Google Drive
from google.colab import drive
import os
drive.mount('/content/drive')
# Verify the path
# CHANGE THIS PATH if your folder is named differently
base_dir = '/content/drive/My Drive/Raw_Dataset'
if os.path.exists(base_dir):
    print(f"Dataset found at: {base_dir}")
    print("Folders:", os.listdir(base_dir))
else:
    print(f"ERROR: Dataset not found at {base_dir}. Please check your Drive structure.")

Mounted at /content/drive
Dataset found at: /content/drive/My Drive/Raw_Dataset
Folders: ['Reaper Marks Defect', 'Pin Head Bubbles Defect', 'Good Quality with No Defects']


In [3]:
# @title Data Loading with Automatic Resizing
import tensorflow as tf
import numpy as np
# Configuration
# MobileNetV2 expects 224x224.
# We resize from 3000x4000 to 224x224 HERE to save RAM.
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
SEED = 42
print("Loading Training Set...")
train_ds = tf.keras.utils.image_dataset_from_directory(
    base_dir,
    validation_split=0.2,
    subset="training",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='int' # 0, 1, 2, 3
)
print("\nLoading Validation Set...")
val_ds = tf.keras.utils.image_dataset_from_directory(
    base_dir,
    validation_split=0.2,
    subset="validation",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='int'
)
class_names = train_ds.class_names
print("\nClasses found:", class_names)
# EXPECTED: ['Reaper Marks Defect', 'Pin Head Bubbles Defect', 'Good Quality with No Defects']

Loading Training Set...
Found 360 files belonging to 3 classes.
Using 288 files for training.

Loading Validation Set...
Found 360 files belonging to 3 classes.
Using 72 files for validation.

Classes found: ['Good Quality with No Defects', 'Pin Head Bubbles Defect', 'Reaper Marks Defect']


In [4]:
# @title Configure Dataset Performance
AUTOTUNE = tf.data.AUTOTUNE
# Cache validation set for speed
# Shuffle training set for better accuracy
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [5]:
import tensorflow as tf
from tensorflow.keras import layers
# import matplotlib.pyplot as plt # Removed for now

# 🔹 Allow only valid image extensions
VALID_EXTENSIONS = (".png", ".jpg", ".jpeg")

def is_valid_image(file_path):
    file_path = tf.strings.lower(file_path)
    return tf.reduce_any(
        [tf.strings.regex_full_match(file_path, f".*\\{ext}") for ext in VALID_EXTENSIONS]
    )

# Define num_classes and one_hot_encode function
num_classes = len(class_names)

def one_hot_encode(image, label):
    label = tf.cast(label, tf.int32)
    return image, tf.one_hot(label, num_classes)

# Define MobileNetV2 preprocessing function
def preprocess_mobilenet(image, label):
    image = tf.keras.applications.mobilenet_v2.preprocess_input(image)
    return image, label

# 🔹 Data augmentation layer (defined once)
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.2),
])

# Function to apply augmentation to a single (image, label) pair
def apply_augmentation(image, label):
    return data_augmentation(image, training=True), label

# --- Dataset pipeline setup ---
# Filter, one-hot encode, then augmentation (for train) and preprocessing (for both)
train_ds = train_ds.filter(
    lambda image, label: tf.equal(image.shape[-1], 3)
).ignore_errors().map(one_hot_encode).map(apply_augmentation).map(preprocess_mobilenet)

val_ds = val_ds.filter(
    lambda image, label: tf.equal(image.shape[-1], 3)
).ignore_errors().map(one_hot_encode).map(preprocess_mobilenet)

# Visualization block removed temporarily.

In [6]:
# @title Build MobileNetV2 Model (Overfitting Fixed)

import tensorflow as tf
from tensorflow.keras import layers, regularizers

# 1. Base Model (Pre-trained)
base_model = tf.keras.applications.MobileNetV2(
    input_shape=IMG_SIZE + (3,),
    include_top=False,
    weights="imagenet"
)

# 🔒 Freeze most layers, allow fine-tuning of top layers only
for layer in base_model.layers[:-40]:
    layer.trainable = False
for layer in base_model.layers[-40:]:
    layer.trainable = True

# 2. Classification Head
inputs = tf.keras.Input(shape=IMG_SIZE + (3,))
# Data augmentation and MobileNetV2 preprocessing are now handled in the dataset pipeline.
x = base_model(inputs, training=True)

x = layers.GlobalAveragePooling2D()(x)

# 🔹 Stronger regularization
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.5)(x)

# 🔹 Dense layer with L2 regularization
x = layers.Dense(
    128,
    activation="relu",
    kernel_regularizer=regularizers.l2(0.01) # Restored original value
)(x)

x = layers.Dropout(0.5)(x)

outputs = layers.Dense(
    len(class_names),
    activation="softmax",
    kernel_regularizer=regularizers.l2(0.001)
)(x)

model = tf.keras.Model(inputs, outputs)

# 3. Compile with LOWER learning rate + label smoothing
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=["accuracy"]
)

model.summary()

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1280)           │         5,120 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,427,459 (9.26 MB)

 Trainable params: 1,848,451 (7.05 MB)

 Non-trainable params: 579,008 (2.21 MB)

In [9]:
# @title Phase 1: Train Top Layers
EPOCHS_INITIAL = 10
history = model.fit(
    train_ds,
    epochs=EPOCHS_INITIAL,
    validation_data=val_ds
)

Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 14s 3s/step - accuracy: 0.8818 - loss: 2.9701 - val_accuracy: 0.6806 - val_loss: 3.2252
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 13s 3s/step - accuracy: 0.9052 - loss: 2.9167 - val_accuracy: 0.7222 - val_loss: 3.2060
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 24s 5s/step - accuracy: 0.9247 - loss: 2.9031 - val_accuracy: 0.7222 - val_loss: 3.1762
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 13s 3s/step - accuracy: 0.9447 - loss: 2.8402 - val_accuracy: 0.7778 - val_loss: 3.1626
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 13s 3s/step - accuracy: 0.9359 - loss: 2.8508 - val_accuracy: 0.7778 - val_loss: 3.1681
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 13s 3s/step - accuracy: 0.9048 - loss: 2.8518 - val_accuracy: 0.7778 - val_loss: 3.1517
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 31s 3s/step - accuracy: 0.8887 - loss: 2.8970 - val_accuracy: 0.7778 - val_loss: 3.1332
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 13s 3s/step - accuracy: 0.9522 - loss: 2.7878 - val_accuracy: 0.7778 - val_loss: 3.1252
Epoch 9/

In [11]:
# @title Phase 2: Fine Tuning (Validation Accuracy Fix)

# 🔓 Enable fine-tuning
base_model.trainable = True

# Fine-tune only LAST 20 layers (safer)
fine_tune_at = len(base_model.layers) - 20

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

# ❗ Freeze BatchNormalization layers (CRITICAL)
for layer in base_model.layers:
    if isinstance(layer, tf.keras.layers.BatchNormalization):
        layer.trainable = False

# 🔁 Recompile with SAFE fine-tuning LR
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=["accuracy"]
)

model.summary()

# 🔢 Fine-tuning epochs
EPOCHS_FINE = 15
total_epochs = EPOCHS_INITIAL + EPOCHS_FINE

# 📉 Callbacks to help validation accuracy
callbacks_fine = [
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.3,
        patience=2,
        min_lr=1e-6
    ),
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=5,
        restore_best_weights=True
    )
]

# 🚀 Continue training
history_fine = model.fit(
    train_ds,
    epochs=total_epochs,
    initial_epoch=history.epoch[-1],
    validation_data=val_ds,
    callbacks=callbacks_fine
)

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1280)           │         5,120 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,427,459 (9.26 MB)

 Trainable params: 1,361,795 (5.19 MB)

 Non-trainable params: 1,065,664 (4.07 MB)

Epoch 10/25
5/5 ━━━━━━━━━━━━━━━━━━━━ 22s 3s/step - accuracy: 0.8316 - loss: 2.8867 - val_accuracy: 0.7361 - val_loss: 3.2459 - learning_rate: 3.0000e-05
Epoch 11/25
5/5 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 0.8767 - loss: 2.8179 - val_accuracy: 0.7778 - val_loss: 3.2416 - learning_rate: 3.0000e-05
Epoch 12/25
5/5 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy: 0.9320 - loss: 2.7736 - val_accuracy: 0.7778 - val_loss: 3.2475 - learning_rate: 3.0000e-05
Epoch 13/25
5/5 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 0.8934 - loss: 2.8536 - val_accuracy: 0.7361 - val_loss: 3.2282 - learning_rate: 3.0000e-05
Epoch 14/25
5/5 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 0.9056 - loss: 2.8349 - val_accuracy: 0.7361 - val_loss: 3.2000 - learning_rate: 3.0000e-05
Epoch 15/25
5/5 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 0.8910 - loss: 2.7828 - val_accuracy: 0.8056 - val_loss: 3.1720 - learning_rate: 3.0000e-05
Epoch 16/25
5/5 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy: 0.9450 - loss: 2.7821

In [12]:
# @title Evaluate Model
loss, accuracy = model.evaluate(val_ds)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")
# Save the Keras model
model.save('rubber_classifier_model.keras')

3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 632ms/step - accuracy: 0.7934 - loss: 3.0579
Validation Accuracy: 80.56%


In [13]:
# @title Convert to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
# Optimization: Quantization (Float16)
# Reduces size by 50% with negligible accuracy loss
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
tflite_model = converter.convert()
# Save the file
tflite_path = '/content/drive/My Drive/rubber_defect_classifier1.tflite'
with open(tflite_path, 'wb') as f:
  f.write(tflite_model)
print(f"Model saved to Drive: {tflite_path}")
# Also save labels for the app
import json
labels_path = '/content/drive/My Drive/labels1.json'
with open(labels_path, 'w') as f:
    json.dump(class_names, f)
print(f"Labels saved to Drive: {labels_path}")

Saved artifact at '/tmp/tmpj26f29za'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor_159')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  140250547635984: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140250547638096: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140250547638288: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140250547637520: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140250547636752: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140250547637712: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140250547640208: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140250547640400: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140250547639248: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140250547637136: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1402505476

In [14]:
import tensorflow as tf
model = tf.keras.models.load_model('rubber_classifier_model.keras')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
context = converter.convert()
with open('rubber_defect_classifier.tflite', 'wb') as f:
  f.write(context)
# Then download the new .tflite file from the files sidebar

Saved artifact at '/tmp/tmpzhb9o_b9'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer_2')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  140249888095504: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140249888096080: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140249888094352: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140249888095888: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140249888094736: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140249888095120: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140249888094160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140249888686544: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140249888094928: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140249888094544: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1402498886863

In [15]:
# @title Sample Image Prediction

import tensorflow as tf
import numpy as np

# Load the saved Keras model
model = tf.keras.models.load_model('rubber_classifier_model.keras')

# Define class names (from earlier in the notebook)
class_names = ['Good Quality with No Defects', 'Pin Head Bubbles Defect', 'Reaper Marks Defect']

# Sample image path - replace with an actual image path from your dataset
# Example: Use one from the validation set or any test image in Google Drive
sample_image_path = '/content/drive/My Drive/Raw_Dataset/Good Quality with No Defects/image183.jpg'  # Replace with a real image file name

# Load the image and resize to 224x224
img = tf.keras.utils.load_img(sample_image_path, target_size=(224, 224))

# Convert to array and add batch dimension
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)  # Create batch axis

# Preprocess for MobileNetV2
img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)

# Make prediction
predictions = model.predict(img_array)
predicted_class_idx = np.argmax(predictions[0])
predicted_class = class_names[predicted_class_idx]
probabilities = predictions[0]

# Output results
print(f"Predicted Class: {predicted_class}")
print(f"Probabilities: {probabilities}")
print(f"Confidence for {predicted_class}: {probabilities[predicted_class_idx] * 100:.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Predicted Class: Good Quality with No Defects
Probabilities: [9.999291e-01 3.419305e-06 6.752334e-05]
Confidence for Good Quality with No Defects: 99.99%


In [21]:
# @title Sample Image Prediction

import tensorflow as tf
import numpy as np

# Load the saved Keras model
model = tf.keras.models.load_model('rubber_classifier_model.keras')

# Define class names (from earlier in the notebook)
class_names = ['Good Quality with No Defects', 'Pin Head Bubbles Defect', 'Reaper Marks Defect']

# Sample image path - replace with an actual image path from your dataset
# Example: Use one from the validation set or any test image in Google Drive
sample_image_path = '/content/drive/My Drive/Raw_Dataset/Reaper Marks Defect/image202.jpeg'  # Replace with a real image file name

# Load the image and resize to 224x224
img = tf.keras.utils.load_img(sample_image_path, target_size=(224, 224))

# Convert to array and add batch dimension
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)  # Create batch axis

# Preprocess for MobileNetV2
img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)

# Make prediction
predictions = model.predict(img_array)
predicted_class_idx = np.argmax(predictions[0])
predicted_class = class_names[predicted_class_idx]
probabilities = predictions[0]

# Output results
print(f"Predicted Class: {predicted_class}")
print(f"Probabilities: {probabilities}")
print(f"Confidence for {predicted_class}: {probabilities[predicted_class_idx] * 100:.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Predicted Class: Reaper Marks Defect
Probabilities: [3.1538526e-04 6.4540443e-05 9.9962008e-01]
Confidence for Reaper Marks Defect: 99.96%
